Pour exécuter les notebooks suivants, si ce n'est pas encore fait, vous devez déployer un modèle qui utilise `text-embedding-ada-002` comme modèle de base et définir le nom du déploiement dans le fichier .env sous le nom `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Ensuite, nous allons charger l'Index d'Embedding dans un DataFrame Pandas. L'Index d'Embedding est stocké dans un fichier JSON appelé `embedding_index_3m.json`. L'Index d'Embedding contient les Embeddings pour chacune des transcriptions YouTube jusqu'à fin octobre 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Ensuite, nous allons créer une fonction appelée `get_videos` qui recherchera dans l'Index d'Embedding la requête. La fonction retournera les 5 vidéos les plus similaires à la requête. La fonction fonctionne comme suit :

1. Tout d'abord, une copie de l'Index d'Embedding est créée.
2. Ensuite, l'Embedding pour la requête est calculé en utilisant l'API d'Embedding OpenAI.
3. Puis, une nouvelle colonne est créée dans l'Index d'Embedding appelée `similarity`. La colonne `similarity` contient la similarité cosinus entre l'Embedding de la requête et l'Embedding de chaque segment vidéo.
4. Ensuite, l'Index d'Embedding est filtré par la colonne `similarity`. L'Index d'Embedding est filtré pour ne conserver que les vidéos ayant une similarité cosinus supérieure ou égale à 0,75.
5. Enfin, l'Index d'Embedding est trié par la colonne `similarity` et les 5 meilleures vidéos sont retournées.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Cette fonction est très simple, elle affiche simplement les résultats de la requête de recherche.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Tout d'abord, l'Index d'Embedding est chargé dans un Dataframe Pandas.  
2. Ensuite, l'utilisateur est invité à saisir une requête.  
3. Puis, la fonction `get_videos` est appelée pour rechercher dans l'Index d'Embedding la requête.  
4. Enfin, la fonction `display_results` est appelée pour afficher les résultats à l'utilisateur.  
5. L'utilisateur est ensuite invité à saisir une autre requête. Ce processus continue jusqu'à ce que l'utilisateur saisisse `exit`.  

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc.fr.png)  

Vous serez invité à saisir une requête. Entrez une requête et appuyez sur Entrée. L'application retournera une liste de vidéos pertinentes par rapport à la requête. L'application retournera également un lien vers l'endroit dans la vidéo où se trouve la réponse à la question.  

Voici quelques requêtes à essayer :  

- Qu'est-ce que Azure Machine Learning ?  
- Comment fonctionnent les réseaux de neurones convolutionnels ?  
- Qu'est-ce qu'un réseau de neurones ?  
- Puis-je utiliser Jupyter Notebooks avec Azure Machine Learning ?  
- Qu'est-ce que ONNX ?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Avertissement** :  
Ce document a été traduit à l’aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d’assurer l’exactitude, veuillez noter que les traductions automatiques peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d’origine doit être considéré comme la source faisant foi. Pour les informations critiques, une traduction professionnelle réalisée par un humain est recommandée. Nous déclinons toute responsabilité en cas de malentendus ou de mauvaises interprétations résultant de l’utilisation de cette traduction.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
